In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas
from numpy.linalg import inv
import random
import csv
import math


cov = []
mean0 = []
mean1 = []
with open('./data/DS1_Cov.txt') as f:
    for line in f:
        to_add = line.split(',')
        to_add = [float(i) for i in to_add[0:len(to_add) - 1]]
        cov.append(to_add)

with open('./data/DS1_m_0.txt') as f:
    for line in f:
        to_add = line.split(',')
        to_add = [float(i) for i in to_add[0:len(to_add) - 1]]
        mean0.append(to_add)

        
with open('./data/DS1_m_1.txt') as f:
    for line in f:
        to_add = line.split(',')
        to_add = [float(i) for i in to_add[0:len(to_add) - 1]]
        mean1.append(to_add)
    

In [2]:
def sampling(data_array):
    np.random.shuffle(data_array)
    test_set = []
    train_set = []
    for row in data_array[0:600]:
        test_set.append(row)
    
    for row in data_array[600:2001]:
        train_set.append(row)
        
    return test_set,train_set
    

In [3]:
positive = np.random.multivariate_normal(mean1[0],cov, 2000)
negative = np.random.multivariate_normal(mean0[0],cov, 2000)

In [4]:
test1,train1 = sampling(positive)
t_1 = [1 for i in range(0,1400)]
test2,train2 = sampling(negative)
t_0 = [0 for i in range(0,1400)]
X=list(train1)
#t = t_1
#t.extend(t_2)
X.extend(train2)
def MLE_estimator(train1, train2, t_0,t_1):
    MLE_mean0 = (1/len(t_0))*(np.matmul(np.transpose(train1),t_0))
    MLE_mean1 = (1/len(t_1))*(np.matmul(np.transpose(train2),t_1))
    pi = len(t_1)/(len(t_1) + len(t_0))
    S_1 = 1/(len(t_1)) * (np.matmul(np.transpose([(i - mean1)[0] for i in train1]),([(i - mean1)[0] for i in train1])))
    #S_1 = positive
    S_2 = 1/(len(t_1)) * (np.matmul(np.transpose([(i - mean0)[0] for i in train2]), [(i - mean0)[0] for i in train2]))

    sigma = (len(t_1)/(len(t_1) + len(t_0)))*S_1 + (len(t_0)/(len(t_1) + len(t_0)))*S_2
    
    omega = np.matmul(inv(sigma), np.transpose(np.subtract(mean1, mean0)))
    omega0 = (-1/2)*np.matmul(np.matmul(mean1,inv(sigma)),np.transpose(mean1)) + (1/2)*np.matmul(np.matmul(mean0,inv(sigma)),np.transpose(mean0))
    return omega0,omega

q2_omega0,q2_omega = MLE_estimator(train1,train2, t_0, t_1)


If entries are negative then, we know that probability of it being positive is less than $\frac{1}{2}$, if entries are positive, the probabilitiy is greater than $\frac{1}{2}$. Hence we can select for positive and negative entries.

In [36]:

pred1 = np.matmul(test1,q2_omega) + q2_omega0
pred2 = np.matmul(test2,q2_omega) + q2_omega0

def count(mat):
    positive = 0
    negative = 0
    for i in mat:
        if (i > 0):
            positive +=1
        else:
            negative +=1
    return positive, negative
print(count(pred1))
print(count(pred2))

accuracy = (count(pred1)[0] + count(pred2)[1])/1200
precision = (count(pred1)[0])/(count(pred1)[0] + count(pred1)[1])
recall = (count(pred1)[0])/(count(pred1)[0] + count(pred2)[0])



print("The accuracy of this classifier is " + str(100*accuracy) + "%")
print("The precision of this classifier is " + str(100*precision) + "%")
print("The recall of this classifier is " + str(100*recall) + "%")
print("The F-measure of this classifier is " + str((2*precision*recall)/(precision + recall)))


(574, 26)
(29, 571)
The accuracy of this classifier is 95.41666666666667%
The precision of this classifier is 95.66666666666667%
The recall of this classifier is 95.19071310116087%
The F-measure of this classifier is 0.9542809642560266


In [6]:
def distance(vector1, vector2):
    out = 0
    for i in range(0,len(vector1)):
        out += pow((vector1[i] - vector2[i]),2)
    return math.sqrt(out)

def knearestneighbourhood(train,point,k):
    distances = []
    length = len(train)
    for x in range(0,length):
        dist = distance(point,X[x])
        if (x < length/2):
            distances.append((dist,1))
        else:
            distances.append((dist,0))
    distances = sorted(distances)[:k]
    frequency_p = 0
    frequency_n = 0 
    
    for dist in distances:
        if dist[1] == 0:
            frequency_n += 1
        elif dist[1] == 1:
            frequency_p += 1
    return 0 if frequency_n>frequency_p else 1


In [7]:
def resultsknearest(test,train,k):
    out = []
    for i in test:
        out.append(knearestneighbourhood(train,i,k))
    return out



In [8]:
positive_values = []
negative_values = []
for k in range(1,13):
    sample1 = resultsknearest(test1, train1,k)
    sample2 = resultsknearest(test2, train2,k)
    positive_values.append(count(sample1))
    negative_values.append(count(sample2))

In [43]:
print(positive_values)
print(negative_values)
def best_fit(positive_values,negative_values):
    
    best_f_measure = -5
    index = 0
    for i in range(0,len(positive_values)):
        accuracy = (positive_values[i][0] + negative_values[i][1])/1200
        precision = (positive_values[i][0])/(positive_values[i][0] + positive_values[i][1])
        recall = (positive_values[i][0])/(positive_values[i][0] + negative_values[i][0])
        F_measure = 2*precision/(precision + recall)
        if (F_measure > best_f_measure):
            best_f_measure = F_measure
            index = i
    accuracy = (positive_values[index][0] + negative_values[index][1])/1200
    precision = (positive_values[index][0])/(positive_values[index][0] + positive_values[index][1])
    recall = (positive_values[index][0])/(positive_values[index][0] + negative_values[index][0])
    F_measure = 2*(precision*recall)/(precision + recall)
    
    return accuracy,precision,recall,F_measure,index

accuracy,precision,recall,F_measure,index = best_fit(positive_values,negative_values)
print("value of k that performs the best is ", index+1)

print("The accuracy of this classifier is " + str(100*accuracy) + "%")
print("The precision of this classifier is " + str(100*precision) + "%")
print("The recall of this classifier is " + str(100*recall) + "%")
print("The F-measure of this classifier is " + str(2*(precision*recall)/(precision + recall)))
    



    



[(288, 312), (461, 139), (324, 276), (435, 165), (324, 276), (405, 195), (306, 294), (394, 206), (306, 294), (383, 217), (313, 287), (385, 215)]
[(310, 290), (450, 150), (313, 287), (406, 194), (308, 292), (396, 204), (320, 280), (381, 219), (305, 295), (391, 209), (326, 274), (374, 226)]
value of k that performs the best is  2
The accuracy of this classifier is 50.916666666666664%
The precision of this classifier is 76.83333333333333%
The recall of this classifier is 50.60373216245884%
The F-measure of this classifier is 0.6101919258769027


In [20]:
q4_cov1 = []
q4_cov2 = []
q4_cov3 = []
q4_c1_mean1 = []
q4_c1_mean2 = []
q4_c1_mean3 = []
q4_c2_mean1 = []
q4_c2_mean2 = []
q4_c2_mean3 = []
probability = [0.1,0.42,0.48]

with open('./data/DS2_Cov1.txt') as f:
    for line in f:
        to_add = line.split(',')
        to_add = [float(i) for i in to_add[0:len(to_add) - 1]]
        q4_cov1.append(to_add)
with open('./data/DS2_Cov2.txt') as f:
    for line in f:
        to_add = line.split(',')
        to_add = [float(i) for i in to_add[0:len(to_add) - 1]]
        q4_cov2.append(to_add)
with open('./data/DS2_Cov3.txt') as f:
    for line in f:
        to_add = line.split(',')
        to_add = [float(i) for i in to_add[0:len(to_add) - 1]]
        q4_cov3.append(to_add)      
with open('./data/DS2_c1_m1.txt') as f:
    for line in f:
        to_add = line.split(',')
        to_add = [float(i) for i in to_add[0:len(to_add) - 1]]
        q4_c1_mean1.append(to_add)
with open('./data/DS2_c1_m2.txt') as f:
    for line in f:
        to_add = line.split(',')
        to_add = [float(i) for i in to_add[0:len(to_add) - 1]]
        q4_c1_mean2.append(to_add)
with open('./data/DS2_c1_m3.txt') as f:
    for line in f:
        to_add = line.split(',')
        to_add = [float(i) for i in to_add[0:len(to_add) - 1]]
        q4_c1_mean3.append(to_add)
with open('./data/DS2_c2_m1.txt') as f:
    for line in f:
        to_add = line.split(',')
        to_add = [float(i) for i in to_add[0:len(to_add) - 1]]
        q4_c2_mean1.append(to_add)
with open('./data/DS2_c2_m2.txt') as f:
    for line in f:
        to_add = line.split(',')
        to_add = [float(i) for i in to_add[0:len(to_add) - 1]]
        q4_c2_mean2.append(to_add)        
with open('./data/DS2_c2_m3.txt') as f:
    for line in f:
        to_add = line.split(',')
        to_add = [float(i) for i in to_add[0:len(to_add) - 1]]
        q4_c2_mean3.append(to_add)        
        
sampling1 = np.random.multinomial(2000,probability)
sampling2 = np.random.multinomial(2000,probability)
class1_m1 = np.random.multivariate_normal(q4_c1_mean1[0],q4_cov1, sampling1[0])
class1_m2 = np.random.multivariate_normal(q4_c1_mean2[0],q4_cov2, sampling1[1])
class1_m3 = np.random.multivariate_normal(q4_c1_mean3[0],q4_cov3, sampling1[2])
class2_m1 = np.random.multivariate_normal(q4_c2_mean1[0],q4_cov1, sampling1[0])
class2_m2 = np.random.multivariate_normal(q4_c2_mean2[0],q4_cov2, sampling1[1])
class2_m3 = np.random.multivariate_normal(q4_c2_mean3[0],q4_cov3, sampling1[2])
positive2 = list(class1_m1)
positive2.extend(list(class1_m2))
positive2.extend(list(class1_m3))
negative2 = list(class2_m1)
negative2.extend(list(class2_m2))
negative2.extend(list(class2_m3))

q5_test1, q5_train1 = sampling(positive2)
q5_t1 = [1 for i in range(0,1400)]
q5_test2, q5_train2 = sampling(negative2)
q5_t2 = [0 for i in range(0,1400)]

q5_omega0, q5_omega = MLE_estimator(q5_train1,q5_train2,q5_t1, q5_t2)

In [21]:
q5_pred1 = np.matmul(q5_test1,q5_omega) + q5_omega0
q5_pred2 = np.matmul(q5_test2,q5_omega) + q5_omega0


In [44]:
print(count(q5_pred1))
print(count(q5_pred2))

accuracy = (count(q5_pred1)[0] + count(q5_pred2)[1])/1200
precision = (count(q5_pred1)[0])/(count(q5_pred1)[0] + count(q5_pred1)[1])
recall = (count(q5_pred1)[0])/(count(q5_pred1)[0] + count(q5_pred2)[0])



print("The accuracy of this classifier is " + str(100*accuracy) + "%")
print("The precision of this classifier is " + str(100*precision) + "%")
print("The recall of this classifier is " + str(100*recall) + "%")
print("The F-measure of this classifier is " + str(2*precision/(precision + recall)))


(212, 388)
(217, 383)
The accuracy of this classifier is 49.583333333333336%
The precision of this classifier is 35.333333333333336%
The recall of this classifier is 49.417249417249415%
The F-measure of this classifier is 0.8338192419825073


In [19]:
q5_positive_values = []
q5_negative_values = []
for k in range(1,13):
    sample1 = resultsknearest(q5_test1, q5_train1,k)
    sample2 = resultsknearest(q5_test2, q5_train2,k)
    q5_positive_values.append(count(sample1))
    q5_negative_values.append(count(sample2))

In [14]:
print(q5_positive_values)
print(q5_negative_values)

[(306, 294), (453, 147), (320, 280), (420, 180), (324, 276), (427, 173), (338, 262), (426, 174), (341, 259), (411, 189), (339, 261), (411, 189)]
[(317, 283), (448, 152), (311, 289), (423, 177), (319, 281), (413, 187), (324, 276), (411, 189), (327, 273), (411, 189), (355, 245), (415, 185)]


In [45]:
q5_accuracy,q5_precision,q5_recall,q5_F_measure,q5_index = best_fit(q5_positive_values,q5_negative_values)
print("value of k that performs the best is ", q5_index+1)
print("The accuracy of this classifier is " + str(100*q5_accuracy) + "%")
print("The precision of this classifier is " + str(100*q5_precision) + "%")
print("The recall of this classifier is " + str(100*q5_recall) + "%")
print("The F-measure of this classifier is " + str(2*(q5_precision*q5_recall)/(q5_precision + q5_recall)))


value of k that performs the best is  2
The accuracy of this classifier is 52.33333333333333%
The precision of this classifier is 78.16666666666666%
The recall of this classifier is 51.53846153846153%
The F-measure of this classifier is 0.6211920529801324


We can see there is little to no difference in performace between these two data